In [1]:
import numpy as np
import pandas as pd
import glob
from astropy.table import Table
import matplotlib.pyplot as plt
import json 
import collections
import astropy

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz

from astropy.io import fits
from astropy.io import ascii

In [33]:
from astroplan.plots import plot_airmass
from astroplan import Observer
from astroplan import FixedTarget
from astroplan import (AltitudeConstraint, AirmassConstraint,
                       AtNightConstraint)
from astroplan import is_observable, is_always_observable, months_observable
observer = Observer.at_site('palomar')

In [2]:
locations = Table.from_pandas(pd.read_csv("options.csv"))

In [6]:
ascii.write(locations, "options.ipac", format= "ipac")

In [12]:
wise_crossmatch= ascii.read("wise_allwise.allwise_p3as_psd_28028.tbl", format = "ipac")

In [17]:
wise_crossmatch[0]

cntr_01,dist_x,pang_x,gaiaedr3_01,ra_01,dec_01,designation,ra,dec,sigra,sigdec,sigradec,w1mpro,w1sigmpro,w1snr,w1rchi2,w2mpro,w2sigmpro,w2snr,w2rchi2,w3mpro,w3sigmpro,w3snr,w3rchi2,w4mpro,w4sigmpro,w4snr,w4rchi2,nb,na,w1sat,w2sat,w3sat,w4sat,pmra,sigpmra,pmdec,sigpmdec,cc_flags,ext_flg,var_flg,ph_qual,moon_lev,w1nm,w1m,w2nm,w2m,w3nm,w3m,w4nm,w4m
,arcsec,deg,,,,,deg,deg,arcsec,arcsec,arcsec,mag,mag,,,mag,mag,,,mag,mag,,,mag,mag,,,,,,,,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,
int32,float64,float64,str19,float64,float64,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,int32,float64,float64,float64,float64,int32,int32,int32,int32,str4,int32,str4,str4,str4,int32,int32,int32,int32,int32,int32,int32,int32
1,0.199751,33.442496,1375814952762454272,233.45602,37.99106,J153349.45+375927.9,233.4560588,37.9911063,0.0482,0.0479,-0.0095,13.763,0.024,45.6,1.466,13.785,0.03,36.1,0.9866,12.097,0.212,5.1,1.008,9.38,0.421,2.6,1.049,1,0,0.0,0.0,0.0,0.0,25,48,12,51,0000,0,97nn,AABC,0011,63,63,63,63,2,33,0,33


In [20]:
wise_crossmatch_nonan = wise_crossmatch[np.where(wise_crossmatch["dist_x"] > 0)]

In [24]:
wise_crossmatch_nonan["w1-w2"] = wise_crossmatch_nonan["w1mpro"] - wise_crossmatch_nonan["w2mpro"]

In [25]:
wise_crossmatch_nonan["w3-w4"] = wise_crossmatch_nonan["w3mpro"] - wise_crossmatch_nonan["w4mpro"]

In [29]:
objects_of_interest = wise_crossmatch_nonan[np.where(wise_crossmatch_nonan["w1-w2"] > 0)]

In [42]:
objects_of_interest.to_pandas().to_csv("objects_of_interest.csv", index = False)

In [31]:
c = SkyCoord(ra = objects_of_interest["ra"], dec = objects_of_interest["dec"], frame="icrs", unit="deg")

In [32]:
utcoffset = -7*u.hour
time = Time('2023-05-06 23:30:00') - utcoffset
time_range = Time(['2023-05-06 19:00:00', '2023-05-07 07:00:00']) - utcoffset
delta_t = time_range[1] - time_range[0]
observe_time = time_range[0] + delta_t*np.linspace(0, 1, 75)

In [34]:
objects = []
for i, j in zip(objects_of_interest["gaiaedr3_01"], c):
    objects.append(FixedTarget(name = i, coord = j))
objects_np = np.asarray(objects)

In [38]:
objects_np[0].name

'4102198997623267456'

In [41]:
for i in objects_np:
    plot_airmass(i, observer, observe_time, brightness_shading=True, altitude_yaxis=True)
    plt.savefig("airmass_diagrams/" + str(i.name) + ".png", dpi = 600)
    plt.clf()
    plt.close()

c:\Users\20xha\anaconda3\envs\ay-ge_107\lib\site-packages\astroplan\plots\time_dependent.py:194: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(timetoplot.plot_date, masked_airmass, label=target_name, **style_kwargs)
c:\Users\20xha\anaconda3\envs\ay-ge_107\lib\site-packages\astroplan\plots\time_dependent.py:194: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(timetoplot.plot_date, masked_airmass, label=target_name, **style_kwargs)
c:\Users\20xha\anaconda3\envs\ay-ge_107\lib\site-packages\astroplan\plots\time_dependent.py:194: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-" (-> linestyle='-'). The keyword argument will take precedence.
  ax.plot_date(timetoplot.plo